In [ ]:
import numpy as np
import pandas as pd
import openpyxl
import os 
import xlrd
import datetime
# Setting Directory
os.chdir('C:/Users/TAM/Documents/Vu Quang Nguyen/VACP/')

In [ ]:
#BAN HANG
URL_ban="../VACP/Export BRAVO/Sổ chi tiết bán hàng.xls"  
data_banhang=pd.read_excel(URL_ban,skipfooter=1,header=6).rename(
    columns={
        "Unnamed: 0":"Ngày bán",
        "Unnamed: 1":"ID Order",
        "Unnamed: 2":"ID Khách hàng",
        "Unnamed: 3":"Hãng SX",
        "Unnamed: 5":"ID",
        "Unnamed: 7":"Tên hàng",
        "Tiền bán":"Thành tiền",
        "Unnamed: 19":"Nhân viên",
        "Unnamed: 22":"ID Nhóm hàng"

    }
)
data_banhang=data_banhang[["Ngày bán","ID Order","ID Khách hàng","ID Nhóm hàng","Nhân viên","ID","Tên hàng","Số lượng","Giá bán","Thành tiền"]]


#CONG NO
data_congno=pd.read_excel("../VACP/Export BRAVO/Báo cáo công nợ phải thu theo hạn thanh toán.xls",skipfooter=1,header=6).rename(
    columns={
        "Mã":"ID Khách hàng",
        "Số":"ID Order",
        "Unnamed: 15":"Tiền thu được"
    }
).dropna(subset=["Ngày","ID Order"]).reset_index(drop=True)


# Data NHẬP MUA BÁN
data_nhap_mua_ban=pd.read_excel("../VACP/Input & Export Campaign/Report_Nhap_Mua_Ban.xlsx",sheet_name="Nhập Mua Bán",usecols="B,F:G,S:V")

# DIM_Customer
dim_customer=pd.read_excel("../VACP/Export BRAVO/Data_KH_VAPC.xlsx",sheet_name='Data_KH')


In [ ]:
#Tổng hợp đơn hàng và tính Margin, Tiền thu được
data_banhang_full=data_banhang.merge(data_nhap_mua_ban[["ID","Giá vốn"]],on="ID",how='left')
# Tính Profit
data_banhang_full["Profit"]=(data_banhang_full["Thành tiền"]-data_banhang_full["Giá vốn"]*data_banhang_full["Số lượng"]) 
# Tính margin 
data_banhang_full["Margin"]=data_banhang_full['Profit']/data_banhang_full["Thành tiền"]
# Tổng hợp
data_banhang_full=data_banhang_full.groupby(["Ngày bán","ID Order","ID Khách hàng"],as_index=False).agg({
            'Thành tiền':'sum',
            'Profit':'sum',
            'Margin':'mean'})
# Import Công nợ

data_banhang_full=data_banhang_full.merge(data_congno[["ID Order","ID Khách hàng","Tiền thu được"]],on=['ID Order','ID Khách hàng'],how='left').sort_values(["Ngày bán","Tiền thu được"],ascending=False).reset_index(drop=True)

data_banhang_full['Tiền thu được'] = data_banhang_full['Tiền thu được'].replace(np.nan, 0)

# Khả năng thanh toán = Tiền thu được / Công nợ
data_banhang_full["Cash ratio"]=data_banhang_full["Tiền thu được"]/(data_banhang_full["Thành tiền"])
#SHOW

data_banhang_full.head()



In [ ]:
data_banhang_full.loc[(data_banhang_full["ID Khách hàng"]=='CHUBASG') & (data_banhang_full["Ngày bán"].dt.strftime("%Y-%m-%d")=='2022-06-17')]

In [ ]:
# Tổng hợp Doanh số theo ngày
data_sale=data_banhang_full.groupby(["Ngày bán","ID Khách hàng"],as_index=False).agg({
            'Thành tiền':'sum',
            'Tiền thu được':'sum',
            'Profit':'sum',
            'Margin':'mean',
            'Cash ratio':'mean',
            "ID Order": lambda x: x.nunique()}).rename(columns={"ID Order":"Tổng đơn hàng","Thành tiền":"Doanh số"}).reset_index(drop=True)

# data_sale=data_sale.merge(dim_customer[["ID Khách hàng","Phân hạng","Nhân viên phụ trách","Tên khách hàng","Tỉnh","Vùng miền"]],on="ID Khách hàng",how="left")
data_sale.head()

In [ ]:
# Lọc mẫu dữ liệu
data_sale.loc[(data_sale['ID Khách hàng']=='ANHCUONG') & (data_sale["Ngày bán"].dt.strftime('%Y-%m-%d') == '2022-11-19')]

In [ ]:
data_sale.loc[(data_sale["ID Khách hàng"]=='TRITIN') & (data_sale["Ngày bán"].dt.strftime("%Y-%m-%d")=='2022-07-01')]

In [ ]:
# Data Sale năm 2022

data_sale_total=data_sale.loc[data_sale["Ngày bán"].dt.strftime("%Y")=='2022'].groupby("ID Khách hàng",as_index=False).agg({
    'Doanh số':'sum',
    'Tiền thu được':'sum',
    'Profit':'sum',
    'Tổng đơn hàng':'sum',
    'Cash ratio':'mean'      
}).sort_values(["Doanh số","Profit"],ascending=False).reset_index(drop=True)

data_sale_total["Gross Profit Margin"]=data_sale_total["Profit"]/data_sale_total["Doanh số"]


data_sale_total

In [ ]:
data_sale_filter=data_sale_total.query('`Gross Profit Margin`>=0.02 & `Tổng đơn hàng`>=10 & `Cash ratio`>=0.7').sort_values(["Doanh số","Gross Profit Margin"],ascending=False).reset_index(drop=True)
data_sale_filter=data_sale_filter.merge(dim_customer[["ID Khách hàng","Phân hạng","Nhân viên phụ trách","Tên khách hàng","Tỉnh","Vùng miền"]],on="ID Khách hàng",how="left")
data_sale_filter=data_sale_filter.loc[0:149,:]
data_sale_filter

In [ ]:
# EXPORT FILE DATA
with pd.ExcelWriter('../VACP/Input & Export Campaign/Bao_Cao_Doanh_So_Khach_Hang.xlsx') as writer:
    data_sale.to_excel(writer,sheet_name="Sales Daily")
    data_banhang_full.to_excel(writer,sheet_name="Data")
    data_sale_filter.to_excel(writer,sheet_name="Top 150")
    # data_report.to_excel(writer,sheet_name="Tổng hợp")
    # data_danhmuchanghoa.to_excel(writer,sheet_name="Danh mục hàng")
    # count_danh_muc.to_excel(writer,sheet_name="đến số lượng")
